In [78]:
import pandas as pd
import nltk
import re
import math
import timeit
from numpy import nan
import sqlalchemy

In [79]:
path = "/Users/nathan/Documents/Stats/Data Analysis/"
destination = "/Users/nathan/Documents/Stats/SignalDataScience/finalproject/datafiles/"
global debug 
debug = False
database = destination + "reviews.db"
disk_engine = sqlalchemy.create_engine('sqlite:///' + database)

In [80]:
def query_db(query, disk_engine):  
    start_time = timeit.default_timer()
    df = pd.read_sql_query(query, disk_engine)
    time_e = round((int(timeit.default_timer() - start_time))/60,2)
    print("Query time: ", time_e, "minutes")
    return df

In [89]:
# Make list of target product asins
query = """
SELECT x.asin, AVG(x.overall) as avg_rating FROM (
 SELECT asin, COUNT(asin) as cnt, overall FROM review_full ) x
 WHERE cnt>=1 
 GROUP BY asin
"""

target_df = query_db(query, disk_engine)
print(target_df.head())


Query time:  0.0 minutes
         asin  avg_rating
0  0071363424         5.0


In [87]:
# make review_subset table with first 20 reviews for each target asin
asin_count = 0
for asin in target_df['asin'].tolist():
    query = "SELECT * FROM review_full WHERE asin = '" + asin + "' ORDER BY unixReviewTime ASC LIMIT 20"
    temp_df = query_db(query, disk_engine)
    temp_df.drop('helpful', axis =1, inplace=True)
    avg_rating = target_df.ix[target_df['asin']==asin, 'avg_rating']
    temp_df['avg_rating'] = float(avg_rating)
    print(temp_df.head())
    if debug == True:
        temp_df.to_sql('review_subset',
                        disk_engine,
                        if_exists='replace',
                        index= False,
                        dtype={'asin':sqlalchemy.types.NVARCHAR(length=10),
                         'overall':sqlalchemy.types.INTEGER(),
                         'reviewTime':sqlalchemy.NVARCHAR(length=10),
                         'reviewerID':sqlalchemy.types.NVARCHAR(length=14),
                         'unixReviewTime':sqlalchemy.types.INTEGER(),
                         'reviewerName':sqlalchemy.types.NVARCHAR(length=30),
                         'vader':sqlalchemy.types.Float(precision=4, asdecimal=True),
                         'avg_rating':sqlalchemy.types.Float(precision=3, asdecimal=True)})
    else:
        temp_df.to_sql('review_subset',
                        disk_engine,
                        if_exists='append',
                        index= False,
                        dtype={'asin':sqlalchemy.types.NVARCHAR(length=10),
                         'overall':sqlalchemy.types.INTEGER(),
                         'reviewTime':sqlalchemy.NVARCHAR(length=10),
                         'reviewerID':sqlalchemy.types.NVARCHAR(length=14),
                         'unixReviewTime':sqlalchemy.types.INTEGER(),
                         'reviewerName':sqlalchemy.types.NVARCHAR(length=30),
                         'vader':sqlalchemy.types.Float(precision=4, asdecimal=True),
                         'avg_rating':sqlalchemy.types.Float(precision=3, asdecimal=True)})
    asin_count += 1

print(asin_count, "products included")

Query time:  0.0 minutes
         asin  overall   reviewTime      reviewerID  unixReviewTime  \
0  0071317791      1.0  03 25, 2013  A26VWCY8WQ0HK2      1364169600   
1  0071317791      5.0  10 12, 2013  A1CLJZB7Q79SLV      1381536000   

  reviewerName  vader  avg_rating  
0         None  0.000         1.0  
1         None  0.068         1.0  
1 products included


In [88]:
# Export review_subset left join on reviewer_scores to csv

query = """
SELECT * FROM (review_subset a
LEFT JOIN (SELECT reviewerID, SUM(reviewerScore)
FROM reviewer_scores
GROUP BY reviewerID
) u
ON a.reviewerID = u.reviewerID)
"""
final_df = query_db(query, disk_engine)
print(final_df.head())
final_df.to_csv(destination + "reviewsummary.csv")

Query time:  0.03 minutes
         asin  overall   reviewTime      reviewerID  unixReviewTime  \
0  0062292129        5  05 21, 2013   AJCHYQCYUGFH5      1369094400   
1  0062292129        5  05 21, 2013    AS8AS8YOZNDC      1369094400   
2  0062292129        5  05 21, 2013  A1CHB3TM1KJYRE      1369094400   
3  0062292129        5  05 21, 2013   A8XZHLFEPB5XC      1369094400   
4  0062292129        5  05 21, 2013   AP4F0P1JFJUQL      1369094400   

      reviewerName   vader  avg_rating      reviewerID  SUM(reviewerScore)  
0           Alicia  2.2922         5.0   AJCHYQCYUGFH5                   0  
1  Amazon Customer  2.4674         5.0    AS8AS8YOZNDC                   0  
2  Amazon Customer  0.6810         5.0  A1CHB3TM1KJYRE                   0  
3  Amazon Customer -0.6342         5.0   A8XZHLFEPB5XC                   0  
4            Angie  1.4588         5.0   AP4F0P1JFJUQL                  10  
